In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
from torch.utils.data import DataLoader

import random
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
trainSET = dsets.ImageFolder(
    # root = '업로드한 trainset 위치'
    root = '/content/drive/MyDrive/traing_dataset',
    transform=transforms.ToTensor()
)
testSET = dsets.ImageFolder(
    # root = '업로드한 testset 위치'
    root = '/content/drive/MyDrive/test_dataset',
    transform=transforms.ToTensor()
)

In [ ]:
#파일 개수 확인
print(len(trainSET))
print(len(testSET))

4145
300


In [ ]:
batch_size = 100
train_loader = DataLoader(dataset=trainSET,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True)

In [ ]:
# Model

class CNN(torch.nn.Module):

#CNN 구성
  def __init__(self):
    super(CNN, self).__init__()

    # layer1 input shape = (batch_size, 1, 256, 340)
    #        Conv = (batch_size, 1, 256, 340)
    #        Pool = (batch_size, 1, 128, 170)

    self.layer1 = torch.nn.Sequential(
      torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
      torch.nn.ReLU(),
      torch.nn.MaxPool2d(kernel_size=2, stride=2)
    )

    # layer2 input shape = (batch_size, 1, 128, 170)
    #        Conv = (batch_size, 1, 128, 170)
    #        Pool = (batch_size, 1, 64, 85)

    self.layer2 = torch.nn.Sequential(
      torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
      torch.nn.ReLU(),
      torch.nn.MaxPool2d(kernel_size=2, stride=2)
    )

    # layer3 input shape = (batch_size, 1, 64, 85)
    #        Conv = (batch_size, 1, 64, 85)
    #        Pool = (batch_size, 1, 32, 43)

    self.layer3 = torch.nn.Sequential(
      torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
      torch.nn.ReLU(),
      torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
    )

    # layer4 input shape = (batch_size, 1, 32, 43)
    #        Linear = ()
    self.layer4 = torch.nn.Linear(1*32*43, 3, bias=True)
    torch.nn.init.xavier_uniform_(self.layer4.weight)

# CNN을 실행할 함수
  def forward(self, x):
    ANS = self.layer1(x)
    ANS = self.layer2(ANS)
    ANS = self.layer3(ANS)
    ANS = ANS.view(ANS.size(0), -1) # layer4를 위해 펼쳐준다
    ANS = self.layer4(ANS)
    return ANS



In [ ]:
learning_rate = 0.001
training_epochs = 20

In [ ]:
model = CNN().to(device)

In [ ]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
#train part
total_batch = len(train_loader)
for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in train_loader:
    X = X.to(device)
    Y = Y.to(device)
    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))


RuntimeError: ignored

In [ ]:
with torch.no_grad():
    X_test = trainSET.data.view(len(trainSET), 1, , ).float().to(device)
    Y_test = trainSET.targets.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

In [ ]:
"""
model.eval()
correct = 0
for data, target in test_loader:
  if use_cuda:
    data = data.cuda()
    target = target.cuda()
  output = model(data)
  prediction = output.data.max(1)[1]
  correct += prediction.eq(target.data).sum()

print('\nTest set: Accuracy: {:.2f}%'.format(100.*correct / len(data_loader.dataset)))
"""